# Tensorflow 2: Model validation, regularization, and callbacks

In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
print(tf.__version__)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense, Flatten, Softmax, Conv2D, MaxPooling2D

## Load the Fashion MNIST data

In [ ]:
# Load the Fashion-MNIST dataset
fashion_mnist_data = tf.keras.datasets.fashion_mnist
(train_images, train_labels), (test_images, test_labels) = fashion_mnist_data.load_data()

labels = [
    'T-shirt/top',
    'Trouser',
    'Pullover',
    'Dress',
    'Coat',
    'Sandal',
    'Shirt',
    'Sneaker',
    'Bag',
    'Ankle boot'
]

train_images = train_images/255.0
test_images = test_images/255.0


## Create a sequential model

In [ ]:
model = Sequential([
    Conv2D(16, (3,3), activation = 'relu', input_shape=(28,28,1)),
    MaxPooling2D((3,3)),
    Flatten(),
    Dense(10, activation='softmax')
])


opt = tf.keras.optimizers.Adam(learning_rate=0.005)
acc = tf.keras.metrics.SparseCategoricalAccuracy()
mae = tf.keras.metrics.MeanAbsoluteError()

model.compile(optimizer=opt, loss = 'sparse_categorical_crossentropy',
              metrics = [acc, mae]
             )


## Starts the training procedure with 20% validation set

In [ ]:
history = model.fit(train_images[..., np.newaxis], train_labels, epochs=10, batch_size=256, validation_split=0.2)


## Using regularization

In [ ]:
model = Sequential([
    Conv2D(16, (3,3), activation = 'relu', input_shape=(28,28,1), 
           kernel_regularizer = tf.keras.regularizers.l2(0.01),
        bias_regularizer  = tf.keras.regularizers.l2(0.001)),
    MaxPooling2D((3,3)),
    Flatten(),
    Dense(10, activation='softmax')
])


opt = tf.keras.optimizers.Adam(learning_rate=0.005)
acc = tf.keras.metrics.SparseCategoricalAccuracy()
mae = tf.keras.metrics.MeanAbsoluteError()

model.compile(optimizer=opt, loss = 'sparse_categorical_crossentropy',
              metrics = [acc, mae]
             )

In [ ]:
history = model.fit(train_images[..., np.newaxis], train_labels, epochs=10, batch_size=256, validation_split=0.2)


## Callbacks in Tensorflow 2

In [ ]:
from tensorflow.keras.callbacks import Callback

In [ ]:
class trainingcallback(Callback):
    def on_train_begin(self, logs = None):
        # Do something at beginning of training
        print("Starting training ...")
    
    def on_train_batch_begin(self, batch, logs = None):
        # Do something at the beginning of every batch iteration
        print("Starting the training of batch {}".format(batch))
        
    def on_epoch_begin(self, epoch, logs=None):
        # Do something at the beginning of every epoch
        print("Starting epoch {}".format(epoch))
        
    def on_train_batch_end(self, batch, logs = None):
        # Do something at the end of every batch iteration
        print("Finished the training of batch {}".format(batch))
     
    def on_epoch_end(self, epoch, logs=None):
        # Do something at the end of every epoch
        print("Finishing epoch {}".format(epoch))
         
    def on_train_end(self, logs = None):
        # Do something at beginning of training
        print("Finished training.")

In [ ]:
model.compile(optimizer = "adam", loss = "mse")
history = model.fit(train_images[..., np.newaxis], train_labels, epochs=10, batch_size=256, validation_split=0.2, callbacks = [trainingcallback()])

## Early stopping using callbacks

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
es = EarlyStopping(monitor='val_accuracy', patience = 5)
history = model.fit(train_images[..., np.newaxis], train_labels, epochs=10, batch_size=256, validation_split=0.2, callbacks = [es()])